In [ ]:
!pip install langchain-community==0.2.15 langchain-chroma==0.1.3 langchain-text-splitters==0.2.2 langchain-huggingface==0.0.3 langchain-groq==0.1.9 unstructured==0.15.0 unstructured[pdf]==0.15.0 nltk==3.8.1


In [ ]:
!apt-get install poppler-utils
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import os
from langchain_community.document_loaders import UnstructuredPDFLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [ ]:
GROQ_API_KEY = "YOURAPIKEY-GROQ"

os.environ["GROQ_API_KEY"]= GROQ_API_KEY

In [ ]:
##loader = DirectoryLoader("data/", glob="./*.pdf", loader_cls=UnstructuredPDFLoader)
documents = loader.load()

ERROR:langchain_community.document_loaders.directory:Error loading file data/hess208.pdf


HTTPError: HTTP Error 403: Forbidden

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/hess208.pdf")
documents = loader.load()


In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
text_chunks = text_splitter.split_documents(documents)

In [ ]:
persist_directory = "doc_chroma_db"

In [ ]:
embeddings=HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vectorstore = Chroma.from_documents(
    documents=text_chunks,
    embedding=embeddings,  # Change 'embedding_function' to 'embedding'
    persist_directory=persist_directory,
)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature = 0.5
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

In [ ]:
query = "What economic impact did the First World War have on India?"

response= qa_chain.invoke({"query":query})

In [ ]:
print(response)

{'query': 'What economic impact did the First World War have on India?', 'result': 'The First World War had a significant economic impact on India. It led to a huge rise in the defence expenditure of the Government of India, which in turn increased taxes on individual incomes and business profits. This caused great difficulties for the common people. On the other hand, business groups reaped fabulous profits from the war. The war created a demand for industrial goods (jute bags, cloth, rails) and caused a sharp rise in prices, making it difficult for people to afford essential goods.', 'source_documents': [Document(metadata={'page': 5, 'source': 'data/hess208.pdf'}, page_content='98\t OUR \tPASTS\t–\tIII\nThe Swadeshi movement sought to oppose British \nrule and encourage the ideas of self-help, swadeshi  \nenterprise, national education, and use of Indian \nlanguages. To fight for swaraj, the radicals advocated \nmass mobilisation and boycott of British institutions \nand goods. Some 

In [ ]:
print(response["result"])

The First World War had a significant economic impact on India. It led to a huge rise in the defence expenditure of the Government of India, which in turn increased taxes on individual incomes and business profits. This caused great difficulties for the common people. On the other hand, business groups reaped fabulous profits from the war. The war created a demand for industrial goods (jute bags, cloth, rails) and caused a sharp rise in prices, making it difficult for people to afford essential goods.


In [ ]:
print(response["source_documents"][0].metadata["source"])

data/hess208.pdf
